In [22]:
# Import API Key
from gptapi_keys import gpt_key, org_key

# Import Dependencies
import os
import openai
openai.organization = org_key
openai.api_key = gpt_key
import pandas as pd
import requests
import json
import base64
from base64 import b64decode
from PIL import Image
import io
from pathlib import Path

output_path = 'output_images/'

In [2]:
# List all availible text models
json_response = openai.Model.list()
json_response = dict(json_response)

# Put models into a DataFrame and show the first 10 in alphabetical order
gpt_models = pd.DataFrame.from_records(json_response['data'])
gpt_models = gpt_models.sort_values(by='id')

gpt_models.head(10)

,id,object,created,owned_by,permission,root,parent
9,ada,model,1649357491,openai,"[{'id': 'modelperm-u0nKN4ub7EVQudgMuvCuvDjc', ...",ada,None
21,ada-code-search-code,model,1651172505,openai-dev,"[{'id': 'modelperm-wa8tg4Pi9QQNaWdjMTM8dkkx', ...",ada-code-search-code,None
26,ada-code-search-text,model,1651172510,openai-dev,"[{'id': 'modelperm-kFc17wOI4d1FjZEaCqnk4Frg', ...",ada-code-search-text,None
37,ada-search-document,model,1651172507,openai-dev,"[{'id': 'modelperm-8qUMuMAbo4EwedbGamV7e9hq', ...",ada-search-document,None
31,ada-search-query,model,1651172505,openai-dev,"[{'id': 'modelperm-b753xmIzAUkluQ1L20eDZLtQ', ...",ada-search-query,None
22,ada-similarity,model,1651172507,openai-dev,"[{'id': 'modelperm-LtSIwCEReeDcvGTmM13gv6Fg', ...",ada-similarity,None
1,babbage,model,1649358449,openai,"[{'id': 'modelperm-49FUp5v084tBB49tC4z8LPH5', ...",babbage,None
5,babbage-code-search-code,model,1651172509,openai-dev,"[{'id': 'modelperm-4qRnA3Hj8HIJbgo0cGbcmErn', ...",babbage-code-search-code,None
10,babbage-code-search-text,model,1651172509,openai-dev,"[{'id': 'modelperm-Lftf8H4ZPDxNxVs0hHPJBUoe', ...",babbage-code-search-text,None
36,babbage-search-document,model,1651172510,openai-dev,"[{'id': 'modelperm-5qFV9kxCRGKIXpBEP75chmp7', ...",babbage-search-document,None


In [19]:
PROMPT = "a black american shorthair cat on top of a couch"
num_images = 1

In [3]:
# Generate Image - Text-to-Image
image_response = openai.Image.create(
    prompt=PROMPT,
    n=num_images,
    size="1024x1024",
    response_format="b64_json",
)

In [20]:
# Create responses folder if not exists
DATA_DIR = Path.cwd() / "responses"
DATA_DIR.mkdir(exist_ok=True)

file_name = DATA_DIR / f"{PROMPT[:5]}-{image_response['created']}.json"

with open(file_name, mode="w", encoding="utf-8") as file:
    json.dump(image_response, file)

In [24]:
JSON_FILE = DATA_DIR / file_name
IMAGE_DIR = Path.cwd() / "images" #/ JSON_FILE.stem
IMAGE_DIR.mkdir(parents=True, exist_ok=True)

with open(JSON_FILE, mode="r", encoding="utf-8") as file:
    response = json.load(file)

for index, image_dict in enumerate(response["data"]):
    image_data = b64decode(image_dict["b64_json"])
    image_file = IMAGE_DIR / f"{JSON_FILE.stem}-{index}.png"
    with open(image_file, mode="wb") as png:
        png.write(image_data)